In [7]:
import json
from kafka import KafkaProducer

# Kafka Producer 설정
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    batch_size=100,
    linger_ms=10,
    acks=1
)

In [32]:
import pytz
import random
import uuid
import pandas as pd
import datetime

sent_events = []

# 1. 이벤트 시간의 시작 지점 설정 (한국 시간 기준으로 설정)
# 예시: 현재 시각으로부터 2일 전, 오전 9시 정각
korea_tz = pytz.timezone('Asia/Seoul')
start_time_kst = korea_tz.localize(datetime.datetime(2025, 6, 28, 16, 26, 0, 0)) # 연, 월, 일, 시, 분, 초, 마이크로초

# 2. 각 이벤트마다 증가시킬 시간 간격 설정
# 예시: 각 이벤트마다 10초씩 증가 (테스트를 위해 짧게 설정)
time_increment = datetime.timedelta(seconds=90)

user_ids = ["111", "222", "333"]
num_events_per_user = 4 # 각 사용자당 보낼 이벤트 개수

current_event_time = start_time_kst

print(f"Starting event time generation from: {current_event_time.isoformat()}")

for _ in range(num_events_per_user * len(user_ids)): # 총 이벤트 개수만큼 반복
    # 사용자 ID를 순환하며 할당
    current_user_id = random.choice(user_ids) # user_ids[i % len(user_ids)] 등으로 순차 할당도 가능

    event = {
        "event_time": current_event_time.isoformat(), # ISO 8601 포맷으로 변환 (예: 2025-06-28T09:00:00+09:00)
        "event_type": random.choice(["view", "cart", "purchase"]),
        "product_id": str(random.randint(10000000, 99999999)),
        "category_id": str(random.randint(1000000000000000000, 9999999999999999999)),
        "category_code": random.choice(["electronics", "beauty", None]),
        "brand": random.choice(["apple", "samsung", "nike"]),
        "price": round(random.uniform(10, 500), 2),
        "user_id": current_user_id,
        "user_session": str(uuid.uuid4())
    }

    # 카프카로 이벤트 전송 (실제 사용 시 주석 해제)
    producer.send("ecommerce-events", value=event)
    sent_events.append(event)

    # 다음 이벤트를 위해 시간 증가
    current_event_time += time_increment

producer.flush() # 모든 메시지가 전송될 때까지 대기
sent_df = pd.DataFrame(sent_events)

Starting event time generation from: 2025-06-28T16:26:00+09:00


In [33]:
# DataFrame 전체 표시 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 전송된 데이터 전체 표시
display(sent_df)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2025-06-28T16:26:00+09:00,cart,96628814,5999828265101493009,electronics,apple,362.95,111,ad453e09-d51f-4bc1-bb1b-65a213d0b4cd
1,2025-06-28T16:27:30+09:00,purchase,13648835,2190161028689234791,beauty,nike,84.42,111,62b78857-4af6-4a08-ac65-1fdfca02440d
2,2025-06-28T16:29:00+09:00,cart,46364723,4202284016997427074,beauty,apple,260.32,222,9d7aba5e-7650-4645-b7bc-1b0c021adba0
3,2025-06-28T16:30:30+09:00,view,75460760,4231635466620517625,None,nike,119.64,222,59156916-18d3-4361-bf25-1472f5fa7f1c
4,2025-06-28T16:32:00+09:00,purchase,29825376,6038347936432067475,electronics,samsung,86.81,333,a0397cb8-d060-42b7-b468-c61a6b89424c
5,2025-06-28T16:33:30+09:00,purchase,24183904,4349055558928834662,electronics,samsung,145.84,111,c070ebe7-e93d-4452-9dc1-4b77d2dde672
6,2025-06-28T16:35:00+09:00,cart,75854501,3214792152358790068,electronics,samsung,241.11,111,4a195777-82d7-426f-b2a6-ed611a631d7e
7,2025-06-28T16:36:30+09:00,purchase,45435615,4886242694021430402,beauty,apple,296.99,111,477899d2-4a86-4eb3-939c-01a3eeda7218
8,2025-06-28T16:38:00+09:00,cart,65684608,6821294205665176485,electronics,nike,190.12,333,fbd3dcf0-65e4-440c-8fe2-fa02b71dcded
9,2025-06-28T16:39:30+09:00,cart,95373212,9341605493601676635,None,apple,76.16,333,02b30964-ee3a-4415-923a-7bb1486a214e


In [34]:
# Redis 연결 및 키 확인
import redis
import json
from pprint import pprint

# Redis 클라이언트 생성
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

try:
    r.ping()
    print('Redis 연결 성공')
except Exception as e:
    print(f"Redis 연결 실패: {e}")


Redis 연결 성공


In [35]:
# 1. 모든 user_features 키 조회
user_keys = r.keys("*")
for key in user_keys[:10]:  # 처음 10개만 표시
    print(f"- {key}")


In [ ]:
r.hgetall("user_features:222")

{}